In [5]:
import pandas as pd
import numpy as np
import random
import time

# Metody doboru sąsiedztwa

In [6]:
def generate_neighbor(solution, neighbor_type='swap'):
    i, j = np.random.choice(len(solution), size=2, replace=False)
    neighbor = solution.copy()
    city1, city2 = neighbor[i], neighbor[j]
    city_pair = [city1, city2]
    if neighbor_type == 'swap':
        # Generuj sąsiada przez zamianę dwóch losowych miast
        neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
    elif neighbor_type == 'insert':
        # Generuj sąsiada przez wstawienie losowego miasta w inne miejsce
        neighbor.insert(j, neighbor.pop(i))
    elif neighbor_type == 'reverse':
        # Generuj sąsiada przez odwrócenie kolejności miast pomiędzy dwoma losowymi indeksami
        if i > j:
            i, j = j, i
        neighbor[i:j + 1] = reversed(neighbor[i:j + 1])
    else:
        raise ValueError("Nieznany rodzaj sąsiedztwa: {}".format(neighbor_type))
    return neighbor, city_pair

# Funkcje trasy

In [7]:
# Stworzenie funkcji ustalenia początkowej trasy
def create_initial_route(distance_matrix):
    route = list(range(len(distance_matrix)))
    random.shuffle(route)
    return route

In [8]:
def calculate_total_distance(solution, distance_matrix):
    total_distance = 0
    num_cities = len(solution)

    for i in range(num_cities - 1):
        total_distance += distance_matrix.at[solution[i], solution[i + 1]]

    # Dodaj odległość z ostatniego do pierwszego miasta
    total_distance += distance_matrix.at[solution[num_cities - 1], solution[0]]

    return total_distance

# Tabu Search

In [10]:
def tabu_search(distance_matrix, initial_solution, tabu_list_size, max_iterations, neighbor_type='swap', gen_neighbor_size=10):
    # Inicjalizacja bieżącego rozwiązania, najlepszego rozwiązania, listy tabu i najlepszego dystansu
    current_solution = initial_solution.copy()
    best_solution = current_solution.copy()
    tabu_list = []
    best_distance = calculate_total_distance(best_solution, distance_matrix)

    # Pętla główna algorytmu Tabu Search
    for _ in range(max_iterations):
        # Generowanie sąsiadów bieżącego rozwiązania
        neighbors = [generate_neighbor(current_solution, neighbor_type) for _ in range(gen_neighbor_size)]

        # Obliczanie dystansów dla kandydatów, którzy nie są na liście tabu
        candidate_distances = [(neighbor, city_pair, calculate_total_distance(neighbor, distance_matrix)) for neighbor, city_pair in neighbors if city_pair not in tabu_list]

        # Jeśli brak kandydatów spoza listy tabu, kontynuuj pętlę
        if not candidate_distances:
            continue

        # Wybieranie najlepszego kandydata
        candidate_solution, city_pair, candidate_distance = min(candidate_distances, key=lambda x: x[2])

        # Aktualizacja najlepszego rozwiązania, jeśli znaleziono lepsze
        if candidate_distance < best_distance:
            best_solution = candidate_solution
            best_distance = candidate_distance

        # Dodanie ruchu do listy tabu
        tabu_list.append(city_pair)
        if len(tabu_list) > tabu_list_size:
            tabu_list.pop(0)  # Usuwanie najstarszego ruchu z listy tabu

    # Zwracanie najlepszego rozwiązania i odpowiadającego mu dystansu
    return best_solution, best_distance


In [11]:
def tabu_search_with_time_limit(distance_matrix, initial_solution, tabu_list_size, max_time_seconds, neighbor_type='swap', gen_neighbor_size=10):
    # Inicjalizacja bieżącego rozwiązania, najlepszego rozwiązania, listy tabu i najlepszego dystansu
    current_solution = initial_solution.copy()
    best_solution = current_solution.copy()
    tabu_list = []
    best_distance = calculate_total_distance(best_solution, distance_matrix)

    # Początkowy czas wykonania algorytmu
    start_time = time.time()

    # Pętla główna algorytmu Tabu Search z ograniczeniem czasowym
    while (time.time() - start_time) < max_time_seconds:
        # Generowanie sąsiadów bieżącego rozwiązania
        neighbors = [generate_neighbor(current_solution, neighbor_type) for _ in range(gen_neighbor_size)]

        # Obliczanie dystansów dla kandydatów, którzy nie są na liście tabu
        candidate_distances = [(neighbor, city_pair, calculate_total_distance(neighbor, distance_matrix)) for neighbor, city_pair in neighbors if city_pair not in tabu_list]

        # Jeśli brak kandydatów spoza listy tabu, kontynuuj pętlę
        if not candidate_distances:
            continue

        # Wybieranie najlepszego kandydata
        candidate_solution, city_pair, candidate_distance = min(candidate_distances, key=lambda x: x[2])

        # Aktualizacja najlepszego rozwiązania, jeśli znaleziono lepsze
        if candidate_distance < best_distance:
            best_solution = candidate_solution
            best_distance = candidate_distance

        # Dodanie ruchu do listy tabu
        tabu_list.append(city_pair)
        if len(tabu_list) > tabu_list_size:
            tabu_list.pop(0)  # Usuwanie najstarszego ruchu z listy tabu

    # Zwracanie najlepszego rozwiązania i odpowiadającego mu dystansu
    return best_solution, best_distance


In [12]:
def tabu_search_multiple_runs(distance_matrix, initial_solution, tabu_list_size, max_iterations, neighbor_type='swap', gen_neighbor_size=10, num_runs=5):
    best_overall_solution = initial_solution.copy()
    best_overall_distance = calculate_total_distance(best_overall_solution, distance_matrix)
    for _ in range(num_runs):
        # Wywołaj funkcję tabu_search i zapisz wyniki
        current_solution, current_distance = tabu_search(distance_matrix, initial_solution, tabu_list_size, max_iterations, neighbor_type, gen_neighbor_size)

        # Sprawdź, czy bieżące rozwiązanie jest lepsze od najlepszego dotychczasowego
        if current_distance < best_overall_distance:
            best_overall_solution = current_solution
            best_overall_distance = current_distance

    # Wydrukuj informacje o najlepszym rozwiązaniu z wszystkich prób
    print("Rozmiar listy tabu: {}, maksymalna powtórzeń: {}, funkcja sąsiedztwa: {}, liczba tworzonych sąsiadów: {}".format(tabu_list_size, max_iterations, neighbor_type, gen_neighbor_size))
    print("Najlepsze rozwiązanie spośród {} prób: {}".format(num_runs, best_overall_solution))
    print("Długość trasy: {}".format(best_overall_distance))

In [13]:
def tabu_search_with_time_limit_multiple_runs(distance_matrix, initial_solution, tabu_list_size, max_time_seconds, neighbor_type='swap', gen_neighbor_size=10, num_runs=5):
    best_overall_solution = initial_solution.copy()
    best_overall_distance = calculate_total_distance(best_overall_solution, distance_matrix)

    for _ in range(num_runs):
        # Wywołaj funkcję tabu_search_with_time_limit i zapisz wyniki
        current_solution, current_distance = tabu_search_with_time_limit(distance_matrix, initial_solution, tabu_list_size, max_time_seconds, neighbor_type, gen_neighbor_size)

        # Sprawdź, czy bieżące rozwiązanie jest lepsze od najlepszego dotychczasowego
        if current_distance < best_overall_distance:
            best_overall_solution = current_solution
            best_overall_distance = current_distance

    # Wydrukuj informacje o najlepszym rozwiązaniu z wszystkich prób
    print("Rozmiar listy tabu: {}, maksymalny czas liczenia: {}, funkcja sąsiedztwa: {}, liczba tworzonych sąsiadów: {}".format(tabu_list_size, max_time_seconds, neighbor_type, gen_neighbor_size))
    print("Najlepsze rozwiązanie spośród {} prób: {}".format(num_runs, best_overall_solution))
    print("Długość trasy: {}".format(best_overall_distance))


## TSP_76

In [14]:
TSP_76 = pd.read_excel('Dane_TSP_76.xlsx', header=None)
# Rozwiazanie poczatkowe wygenerowane przez NN (Nearest Neighbor)
NN_76 = [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 63, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 38, 37, 31, 32, 26, 25, 74, 75, 73]
# Długość trasy:  130921.00457667322

### Sąsiedztwo

In [22]:
tabu_search_multiple_runs(TSP_76, NN_76, 10, 10000, 'swap', 10, 5)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: swap, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 63, 61, 60, 54, 55, 64, 65, 50, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130753.41142129629


Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: swap, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 63, 61, 60, 54, 55, 64, 65, 50, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130753.41142129629

In [23]:
tabu_search_multiple_runs(TSP_76, NN_76, 10, 10000, 'insert', 10, 5)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 63, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130011.77708999392


Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 63, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130011.77708999392

In [24]:
tabu_search_multiple_runs(TSP_76, NN_76, 10, 10000, 'reverse', 10, 5)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 63, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 25, 26, 32, 31, 37, 38, 72, 71, 70, 69, 66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 20, 21, 22, 0, 1, 74, 75, 73]
Długość trasy: 126031.86566023539


Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 63, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 25, 26, 32, 31, 37, 38, 72, 71, 70, 69, 66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 20, 21, 22, 0, 1, 74, 75, 73]
Długość trasy: 126031.86566023539

Wnioski: W generowaniu lepszych wersji rozwiązań, w przypadku dużego zbioru danych, najlepiej sprawdza się funkcja sąsiedztwa reverse. Może to być spowowdowane bardzo dużymi zmianami w kolejności miast.

### Długość listy tabu

In [25]:
tabu_search_multiple_runs(TSP_76, NN_76, 5, 10000, 'insert', 10, 5)

Rozmiar listy tabu: 5, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 63, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130011.77708999392


Rozmiar listy tabu: 5, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 63, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130011.77708999392

In [26]:
tabu_search_multiple_runs(TSP_76, NN_76, 8, 10000, 'insert', 10, 5)

Rozmiar listy tabu: 8, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 63, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130011.77708999392


Rozmiar listy tabu: 8, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 63, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130011.77708999392

In [27]:
tabu_search_multiple_runs(TSP_76, NN_76, 15, 10000, 'insert', 10, 5)

Rozmiar listy tabu: 15, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 63, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130011.77708999392


Rozmiar listy tabu: 15, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 63, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130011.77708999392

In [28]:
tabu_search_multiple_runs(TSP_76, NN_76, 20, 10000, 'insert', 10, 5)

Rozmiar listy tabu: 20, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 63, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130011.77708999392


Rozmiar listy tabu: 20, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 63, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130011.77708999392


### Liczba iteracji

In [29]:
tabu_search_multiple_runs(TSP_76, NN_76, 10, 100, 'insert', 10, 5)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 100, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 63, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130011.77708999392


Rozmiar listy tabu: 10, maksymalna powtórzeń: 100, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 63, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130011.77708999392

In [30]:
tabu_search_multiple_runs(TSP_76, NN_76, 10, 1000, 'insert', 10, 5)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 1000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 63, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130011.77708999392


Rozmiar listy tabu: 10, maksymalna powtórzeń: 1000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 63, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130011.77708999392

In [31]:
tabu_search_multiple_runs(TSP_76, NN_76, 10, 10000,'insert', 10, 5)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 63, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130011.77708999392


Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 63, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130011.77708999392

In [32]:
tabu_search_multiple_runs(TSP_76, NN_76, 10, 100000, 'insert', 10, 5)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 100000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 63, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130011.77708999392


Rozmiar listy tabu: 10, maksymalna powtórzeń: 100000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 63, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130011.77708999392

### Czas liczenia

In [33]:
tabu_search_with_time_limit_multiple_runs(TSP_76, NN_76, 10, 10, 'insert', 10, 5)

Rozmiar listy tabu: 10, maksymalny czas liczenia: 10, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 63, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130011.77708999392


Rozmiar listy tabu: 10, maksymalny czas liczenia: 10, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 63, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130011.77708999392

In [34]:
tabu_search_with_time_limit_multiple_runs(TSP_76, NN_76, 10, 30, 'insert', 10, 5)

Rozmiar listy tabu: 10, maksymalny czas liczenia: 30, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 63, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130011.77708999392


Rozmiar listy tabu: 10, maksymalny czas liczenia: 30, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 63, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130011.77708999392

In [35]:
tabu_search_with_time_limit_multiple_runs(TSP_76, NN_76, 10, 60, 'insert', 10, 5)

Rozmiar listy tabu: 10, maksymalny czas liczenia: 60, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 63, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130011.77708999392


Rozmiar listy tabu: 10, maksymalny czas liczenia: 60, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 63, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130011.77708999392

In [36]:
tabu_search_with_time_limit_multiple_runs(TSP_76, NN_76, 10, 90, 'insert', 10, 5)

Rozmiar listy tabu: 10, maksymalny czas liczenia: 90, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 63, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130011.77708999392


Rozmiar listy tabu: 10, maksymalny czas liczenia: 90, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 63, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130011.77708999392

### Liczba wygenerowanych sąsiedztw

In [37]:
tabu_search_multiple_runs(TSP_76, NN_76, 10, 10000,'insert', 5, 5)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 5
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 63, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130011.77708999392


Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 5
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 63, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130011.77708999392

In [38]:
tabu_search_multiple_runs(TSP_76, NN_76, 10, 10000,'insert', 8, 5)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 8
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 63, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130011.77708999392


Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 8
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 63, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130011.77708999392

In [39]:
tabu_search_multiple_runs(TSP_76, NN_76, 10, 10000,'insert', 15, 5)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 15
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 63, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130011.77708999392


Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 15
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 63, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130011.77708999392

In [40]:
tabu_search_multiple_runs(TSP_76, NN_76, 10, 10000,'insert', 20, 5)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 20
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 63, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130011.77708999392


Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 20
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 63, 38, 37, 31, 32, 26, 25, 74, 75, 73]
Długość trasy: 130011.77708999392

### Wnioski po testach: 
Przeprowadzone testy wskazują, że poza sąsiedztwem lub bardzo małą ilością iteracji, dobór parametrów nie ma większego znaczenia, jeżeli mamy już rozwiązanie w okolicy optymalnego. W testach celowo została użyta funkcja insert doboru sąsiedztwa ze względu na jej przeciętne wyniki, co pozwala na zbadanie wpływu innych parametrów. 

### Poszukiwanie najlepszego rozwiązania dla TSP_76
Do poszukania najlepszego rozwiązanie wykorzystamy trasę wyznaczoną przez algorytm NN oraz parametry, które najlepiej sprawdziły się w wyznaczaniu trasy w testach.

In [11]:
NN_improved = [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 63, 61, 60, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 25, 26, 32, 31, 37, 38, 72, 71, 70, 69, 66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 20, 21, 22, 0, 1, 74, 75, 73]

tabu_search_multiple_runs(TSP_76, NN_improved, 10, 10000, 'reverse', 40, 5)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 40
Najlepsze rozwiązanie spośród 5 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 63, 61, 60, 38, 37, 31, 32, 26, 25, 2, 3, 9, 8, 7, 6, 5, 4, 19, 18, 30, 29, 28, 27, 42, 41, 53, 52, 51, 48, 49, 64, 65, 50, 55, 54, 72, 71, 70, 69, 66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 20, 21, 22, 0, 1, 74, 75, 73]
Długość trasy: 123955.82644412278


In [12]:
NN_improved_2 = [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 63, 61, 60, 38, 37, 31, 32, 26, 25, 2, 3, 9, 8, 7, 6, 5, 4, 19, 18, 30, 29, 28, 27, 42, 41, 53, 52, 51, 48, 49, 64, 65, 50, 55, 54, 72, 71, 70, 69, 66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 20, 21, 22, 0, 1, 74, 75, 73]
# Długość trasy: 123955.82644412278

tabu_search_multiple_runs(TSP_76, NN_improved_2, 10, 10000, 'reverse', 40, 10)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 40
Najlepsze rozwiązanie spośród 10 prób: [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 25, 26, 32, 31, 37, 38, 60, 61, 63, 62, 72, 71, 70, 69, 66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 20, 21, 22, 0, 1, 74, 75, 73]
Długość trasy: 121688.30398853934


In [13]:
NN_improved_3 = [15, 14, 12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 25, 26, 32, 31, 37, 38, 60, 61, 63, 62, 72, 71, 70, 69, 66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 20, 21, 22, 0, 1, 74, 75, 73]
# Długość trasy: 121688.30398853934

tabu_search_with_time_limit_multiple_runs(TSP_76, NN_improved_3, 10, 30, 'reverse', 40, 10)

Rozmiar listy tabu: 10, maksymalny czas liczenia: 30, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 40
Najlepsze rozwiązanie spośród 10 prób: [13, 12, 14, 15, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 25, 26, 32, 31, 37, 38, 60, 61, 63, 62, 72, 71, 70, 69, 66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 20, 21, 22, 0, 1, 74, 75, 73]
Długość trasy: 119798.65413878842


In [14]:
NN_improved_4 = [13, 12, 14, 15, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 25, 26, 32, 31, 37, 38, 60, 61, 63, 62, 72, 71, 70, 69, 66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 20, 21, 22, 0, 1, 74, 75, 73]
# Długość trasy: 119798.65413878842

tabu_search_with_time_limit_multiple_runs(TSP_76, NN_improved_4, 10, 60, 'reverse', 40, 10)

Rozmiar listy tabu: 10, maksymalny czas liczenia: 60, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 40
Najlepsze rozwiązanie spośród 10 prób: [13, 12, 14, 15, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 25, 26, 32, 31, 37, 38, 60, 61, 62, 63, 72, 71, 70, 69, 66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 20, 21, 22, 0, 1, 74, 75, 73]
Długość trasy: 118626.14628956148


In [15]:
NN_improved_5 = [13, 12, 14, 15, 11, 10, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 54, 55, 50, 65, 64, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 3, 2, 25, 26, 32, 31, 37, 38, 60, 61, 62, 63, 72, 71, 70, 69, 66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 20, 21, 22, 0, 1, 74, 75, 73]
# Długość trasy: 118626.14628956148

tabu_search_with_time_limit_multiple_runs(TSP_76, NN_improved_5, 10, 30, 'reverse', 40, 5)

Rozmiar listy tabu: 10, maksymalny czas liczenia: 30, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 40
Najlepsze rozwiązanie spośród 5 prób: [13, 12, 14, 15, 11, 10, 16, 17, 36, 35, 34, 33, 39, 38, 37, 31, 32, 26, 25, 2, 3, 9, 8, 7, 6, 5, 4, 19, 18, 30, 29, 28, 27, 42, 41, 53, 52, 51, 48, 49, 64, 65, 50, 55, 54, 56, 57, 58, 59, 40, 60, 61, 62, 63, 72, 71, 70, 69, 66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 20, 21, 22, 0, 1, 74, 75, 73]
Długość trasy: 117681.76854305725


In [16]:
NN_improved_6 = [13, 12, 14, 15, 11, 10, 16, 17, 36, 35, 34, 33, 39, 38, 37, 31, 32, 26, 25, 2, 3, 9, 8, 7, 6, 5, 4, 19, 18, 30, 29, 28, 27, 42, 41, 53, 52, 51, 48, 49, 64, 65, 50, 55, 54, 56, 57, 58, 59, 40, 60, 61, 62, 63, 72, 71, 70, 69, 66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 20, 21, 22, 0, 1, 74, 75, 73]
# Długość trasy: 117681.76854305725

tabu_search_with_time_limit_multiple_runs(TSP_76, NN_improved_6, 10, 30, 'reverse', 40, 10)

Rozmiar listy tabu: 10, maksymalny czas liczenia: 30, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 40
Najlepsze rozwiązanie spośród 10 prób: [13, 12, 14, 15, 11, 10, 16, 17, 36, 35, 37, 38, 39, 33, 34, 31, 32, 26, 25, 2, 3, 9, 8, 7, 6, 5, 4, 19, 18, 30, 29, 28, 27, 42, 41, 53, 52, 51, 48, 49, 64, 65, 50, 55, 54, 56, 57, 58, 59, 40, 60, 61, 62, 63, 72, 71, 70, 69, 66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 20, 21, 22, 0, 1, 74, 75, 73]
Długość trasy: 116323.24227533051


In [17]:
NN_improved_7 = [13, 12, 14, 15, 11, 10, 16, 17, 36, 35, 37, 38, 39, 33, 34, 31, 32, 26, 25, 2, 3, 9, 8, 7, 6, 5, 4, 19, 18, 30, 29, 28, 27, 42, 41, 53, 52, 51, 48, 49, 64, 65, 50, 55, 54, 56, 57, 58, 59, 40, 60, 61, 62, 63, 72, 71, 70, 69, 66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 20, 21, 22, 0, 1, 74, 75, 73]
# Długość trasy: 116323.24227533051

tabu_search_with_time_limit_multiple_runs(TSP_76, NN_improved_7, 10, 30, 'reverse', 40, 10)

Rozmiar listy tabu: 10, maksymalny czas liczenia: 30, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 40
Najlepsze rozwiązanie spośród 10 prób: [13, 12, 14, 15, 11, 10, 16, 17, 36, 35, 37, 38, 39, 33, 34, 31, 32, 26, 25, 2, 3, 5, 6, 7, 8, 9, 4, 19, 18, 30, 29, 28, 27, 42, 41, 53, 52, 51, 48, 49, 64, 65, 50, 55, 54, 56, 57, 58, 59, 40, 60, 61, 62, 63, 72, 71, 70, 69, 66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 20, 21, 22, 0, 1, 74, 75, 73]
Długość trasy: 115499.25851336036


In [18]:
NN_improved_8 = [13, 12, 14, 15, 11, 10, 16, 17, 36, 35, 37, 38, 39, 33, 34, 31, 32, 26, 25, 2, 3, 5, 6, 7, 8, 9, 4, 19, 18, 30, 29, 28, 27, 42, 41, 53, 52, 51, 48, 49, 64, 65, 50, 55, 54, 56, 57, 58, 59, 40, 60, 61, 62, 63, 72, 71, 70, 69, 66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 20, 21, 22, 0, 1, 74, 75, 73]
# Długość trasy: 115499.25851336036

tabu_search_with_time_limit_multiple_runs(TSP_76, NN_improved_8, 10, 60, 'reverse', 40, 10)

Rozmiar listy tabu: 10, maksymalny czas liczenia: 60, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 40
Najlepsze rozwiązanie spośród 10 prób: [13, 12, 14, 15, 11, 10, 16, 17, 36, 35, 37, 38, 39, 33, 34, 31, 32, 26, 25, 3, 2, 5, 6, 7, 8, 9, 4, 19, 18, 30, 29, 28, 27, 42, 41, 53, 52, 51, 48, 49, 64, 65, 50, 55, 54, 56, 57, 58, 59, 40, 60, 61, 62, 63, 72, 71, 70, 69, 66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 20, 21, 22, 0, 1, 74, 75, 73]
Długość trasy: 114538.96133948538


In [19]:
NN_improved_9 = [13, 12, 14, 15, 11, 10, 16, 17, 36, 35, 37, 38, 39, 33, 34, 31, 32, 26, 25, 3, 2, 5, 6, 7, 8, 9, 4, 19, 18, 30, 29, 28, 27, 42, 41, 53, 52, 51, 48, 49, 64, 65, 50, 55, 54, 56, 57, 58, 59, 40, 60, 61, 62, 63, 72, 71, 70, 69, 66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 20, 21, 22, 0, 1, 74, 75, 73]
# Długość trasy: 114538.96133948538

tabu_search_with_time_limit_multiple_runs(TSP_76, NN_improved_9, 10, 60, 'reverse', 40, 10)

Rozmiar listy tabu: 10, maksymalny czas liczenia: 60, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 40
Najlepsze rozwiązanie spośród 10 prób: [13, 12, 14, 15, 11, 10, 16, 17, 36, 35, 37, 38, 39, 33, 34, 31, 32, 28, 29, 30, 18, 19, 4, 9, 8, 7, 6, 5, 2, 3, 25, 26, 27, 42, 41, 53, 52, 51, 48, 49, 64, 65, 50, 55, 54, 56, 57, 58, 59, 40, 60, 61, 62, 63, 72, 71, 70, 69, 66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 20, 21, 22, 0, 1, 74, 75, 73]
Długość trasy: 113717.58737089993


Wnioski: Algorytm Tabu_Search ma problem ze znalezieniem optimum lokalnego dla dużych zbiorów danych. Wymaga wielokrotnego "restartu" z trasą lepszej jakości uzyskaną w poprzedniej iteracji. Po 9 iteracjach nadal jest w stanie poprawić swój wynik, jednak zysk z tej poprawy maleje, można to zobaczyć na wykresie poprawy wyniku w pliku głównym.

### Ostateczne poszukiwanie rozwiązania - funkcja rekurecnycjna

In [2]:
# Funkcja rekurencyjnie wykorzystująca tabu search do znalezienia najlepszego rozwiązania
def tabu_search_recursive(distance_matrix, initial_solution, tabu_list_size, max_iterations, neighbor_type='swap', gen_neighbor_size=10, num_runs=5):
    best_overall_solution = initial_solution.copy()
    best_overall_distance = calculate_total_distance(best_overall_solution, distance_matrix)
    for _ in range(num_runs):
        # Wywołaj funkcję tabu_search i zapisz wyniki
        current_solution, current_distance = tabu_search(distance_matrix, initial_solution, tabu_list_size, max_iterations, neighbor_type, gen_neighbor_size)

        # Sprawdź, czy bieżące rozwiązanie jest lepsze od najlepszego dotychczasowego
        if current_distance < best_overall_distance:
            best_overall_solution = current_solution
            best_overall_distance = current_distance

    # Wydrukuj informacje o najlepszym rozwiązaniu z wszystkich prób
    print("Rozmiar listy tabu: {}, maksymalna powtórzeń: {}, funkcja sąsiedztwa: {}, liczba tworzonych sąsiadów: {}".format(tabu_list_size, max_iterations, neighbor_type, gen_neighbor_size))
    print("Najlepsze rozwiązanie spośród {} prób: {}".format(num_runs, best_overall_solution))
    print("Długość trasy: {}".format(best_overall_distance))

    # Jeśli znaleziono lepsze rozwiązanie, wywołaj funkcję rekurencyjnie
    if best_overall_solution != initial_solution:
        tabu_search_recursive(distance_matrix, best_overall_solution, tabu_list_size, max_iterations, neighbor_type, gen_neighbor_size, num_runs)

In [15]:
NN_improved_10 = [13, 12, 14, 15, 11, 10, 16, 17, 36, 35, 37, 38, 39, 33, 34, 31, 32, 28, 29, 30, 18, 19, 4, 9, 8, 7, 6, 5, 2, 3, 25, 26, 27, 42, 41, 53, 52, 51, 48, 49, 64, 65, 50, 55, 54, 56, 57, 58, 59, 40, 60, 61, 62, 63, 72, 71, 70, 69, 66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 20, 21, 22, 0, 1, 74, 75, 73]
# Długość trasy: 113717.58737089993

tabu_search_recursive(TSP_76, NN_improved_10, 10, 3000, 'reverse', 40, 10)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 3000, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 40
Najlepsze rozwiązanie spośród 10 prób: [13, 12, 14, 15, 11, 10, 16, 17, 36, 35, 37, 38, 39, 33, 34, 31, 32, 28, 29, 30, 18, 19, 4, 9, 8, 7, 6, 5, 2, 3, 25, 26, 27, 42, 41, 53, 52, 51, 48, 49, 50, 65, 64, 55, 54, 56, 57, 58, 59, 40, 60, 61, 62, 63, 72, 71, 70, 69, 66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 20, 21, 22, 0, 1, 74, 75, 73]
Długość trasy: 113549.994215523
Rozmiar listy tabu: 10, maksymalna powtórzeń: 3000, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 40
Najlepsze rozwiązanie spośród 10 prób: [13, 12, 14, 15, 11, 10, 16, 17, 36, 35, 37, 38, 39, 33, 34, 31, 32, 28, 29, 30, 18, 19, 4, 9, 8, 7, 6, 5, 2, 3, 25, 26, 27, 42, 41, 53, 52, 51, 48, 49, 50, 65, 64, 55, 54, 56, 57, 58, 59, 40, 60, 61, 62, 63, 72, 71, 70, 69, 66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 20, 21, 22, 0, 1, 75, 74, 73]
Długość trasy: 113438.77721469807
Rozmiar listy tabu: 10, maksymalna powtórz

Wnioski: Funkcja rekurencyjna po dodatkowych 4 powtórzeniach znalazła optymalny wynik, oto on:

Rozmiar listy tabu: 10, maksymalna powtórzeń: 3000, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 40
Najlepsze rozwiązanie spośród 10 prób: [13, 73, 74, 75, 1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 69, 70, 71, 72, 63, 62, 61, 60, 40, 59, 58, 57, 56, 54, 55, 64, 65, 50, 49, 48, 51, 52, 53, 41, 42, 27, 26, 25, 3, 2, 5, 6, 7, 8, 9, 4, 19, 18, 30, 29, 28, 31, 32, 34, 33, 39, 38, 37, 35, 36, 17, 16, 10, 11, 15, 14, 12]
Długość trasy: 113403.8307609554